#### FINAL PROJECT PHASE II

### Research Question:

Can we reliably predict a person's drug use based off a person's personality traits? What types of drugs are more likely to be used by certain demographics and personalities?

In this assignment we would like to see if there is a correlation between specific personality traits and drug use, specifically we will look at: euroticism, extraversion, openness to experience, agreeableness, conscientiousness, impulsivity, and sensation seeking. We will train a multivariable regression to see if we can reliably predict a person's drug use based on a person's personality traits.

### Importing:

In [5]:
# imports and settings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from ucimlrepo import fetch_ucirepo 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

### Data Overview:

This dataset contains records for 1885 respondents with the following 12 attributes: personality measurements(neuroticism, extraversion, openness to experience, agreeableness, conscientiousness, impulsivity, sensation seeking), level of education, age, gender, country of residence, and ethnicity. It also contains responses for the responders use of 18 legal and illegal drugs: alcohol, amphetamines, amyl nitrite, benzodiazepine, cannabis, chocolate, cocaine, caffeine, crack, ecstasy, heroin, ketamine, legal highs, LSD, methadone, mushrooms, nicotine and volatile substance abuse and Semerone (fictitious drug to identify overresponders). The responders have selected whether they never used the drug, used it over a decade ago, or in the last decade, year, month, week, or day.

This dataset was accessed from UC Irvine Machine Learning Repository, where it was created by Elaine Fehrman, Vincent Egan, and Evgeny Mirkes. This data set was created to evaluate an individual's risk of drug consumption and misuse based on categorical data. This data collection was independantly created by the authors of the data set. An anonymous online survey methodology from Survey Gizmo was used to collect this data, which could have influenced people's responses because they can choose which questions to answer honestly or not. The people had to take a personality test to get the personality data. The data was analyzed and organized by the three authors listed at the beginning of this paragraph, and the uci machine learning repositiory obtained it for us to get the current form of the data. The people responding were aware of the data collection, as they filled out the form voluntarily, and knew what the data was going to be used for.

Here is the link to the dataset: https://archive.ics.uci.edu/dataset/373/drug+consumption+quantified
and here is the github link to the raw data: https://github.com/uci-ml-repo/ucimlrepo?tab=readme-ov-file

In [10]:
# fetch dataset 
drug_consumption = fetch_ucirepo(id=373) 
  
# data features
drug_consumption.data.features

,age,gender,education,country,ethnicity,nscore,escore,oscore,ascore,cscore,impuslive,ss
0,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084
1,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575
2,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148
3,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084
4,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575
...,...,...,...,...,...,...,...,...,...,...,...,...
1880,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,-1.19430,1.74091,1.88511,0.76096,-1.13788,0.88113,1.92173
1881,-0.95197,-0.48246,-0.61113,-0.57009,-0.31685,-0.24649,1.74091,0.58331,0.76096,-1.51840,0.88113,0.76540
1882,-0.07854,0.48246,0.45468,-0.57009,-0.31685,1.13281,-1.37639,-1.27553,-1.77200,-1.38502,0.52975,-0.52593
1883,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,0.91093,-1.92173,0.29338,-1.62090,-2.57309,1.29221,1.22470


We imported the data frame following the directions from the uci databse website, which imports the dataset as a pandas dataframe. It created a data frame of features shown above, which show the categorical data about the particpant. It also created a data frame of targets shown below, which shows the data about the drug use for each participant.

In [12]:
#data targets
drug_consumption.data.targets 

,alcohol,amphet,amyl,benzos,caff,cannabis,choc,coke,crack,ecstasy,heroin,ketamine,legalh,lsd,meth,mushrooms,nicotine,semer,vsa
0,CL5,CL2,CL0,CL2,CL6,CL0,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,CL5,CL2,CL2,CL0,CL6,CL4,CL6,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,CL6,CL0,CL0,CL0,CL6,CL3,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,CL4,CL0,CL0,CL3,CL5,CL2,CL4,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,CL4,CL1,CL1,CL0,CL6,CL3,CL6,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,CL5,CL0,CL0,CL0,CL4,CL5,CL4,CL0,CL0,CL0,CL0,CL0,CL3,CL3,CL0,CL0,CL0,CL0,CL5
1881,CL5,CL0,CL0,CL0,CL5,CL3,CL4,CL0,CL0,CL2,CL0,CL0,CL3,CL5,CL4,CL4,CL5,CL0,CL0
1882,CL4,CL6,CL5,CL5,CL6,CL6,CL6,CL4,CL0,CL4,CL0,CL2,CL0,CL2,CL0,CL2,CL6,CL0,CL0
1883,CL5,CL0,CL0,CL0,CL6,CL6,CL5,CL0,CL0,CL3,CL0,CL0,CL3,CL3,CL0,CL3,CL4,CL0,CL0


Below, a data frame showing the variables for the data frame is shown, showing the type of each variable, along with the fact that there are no missing values for any of the categories

In [14]:
# variable information 
print(drug_consumption.variables) 

         name     role         type      demographic description units  \
0          id       ID      Integer             None        None  None   
1         age  Feature   Continuous              Age        None  None   
2      gender  Feature   Continuous           Gender        None  None   
3   education  Feature   Continuous  Education Level        None  None   
4     country  Feature   Continuous      Nationality        None  None   
5   ethnicity  Feature   Continuous        Ethnicity        None  None   
6      nscore  Feature   Continuous             None        None  None   
7      escore  Feature   Continuous             None        None  None   
8      oscore  Feature   Continuous             None        None  None   
9      ascore  Feature   Continuous             None        None  None   
10     cscore  Feature   Continuous             None        None  None   
11  impuslive  Feature   Continuous             None        None  None   
12         ss  Feature   Continuous   

##### Data Cleaning:

The first thing we decided to do is focus on the personality aspects, so we delted the columns that were irrelevant to our analysis: level of education, age, gender, country of residence, and ethnicity.

In [17]:
print("Columns before cleaning: ")
print(drug_consumption.data.features.columns)
drug_consumption.data.features = drug_consumption.data.features.drop(['age'], axis=1)
drug_consumption.data.features = drug_consumption.data.features.drop(['gender'], axis=1)
drug_consumption.data.features = drug_consumption.data.features.drop(['education'], axis=1)
drug_consumption.data.features = drug_consumption.data.features.drop(['country'], axis=1)
drug_consumption.data.features = drug_consumption.data.features.drop(['ethnicity'], axis=1)
print(drug_consumption.data.features)
print("Columns after cleaning: ")
print(drug_consumption.data.features.columns)

Columns before cleaning: 
Index(['age', 'gender', 'education', 'country', 'ethnicity', 'nscore',
       'escore', 'oscore', 'ascore', 'cscore', 'impuslive', 'ss'],
      dtype='object')
       nscore   escore   oscore   ascore   cscore  impuslive       ss
0     0.31287 -0.57545 -0.58331 -0.91699 -0.00665   -0.21712 -1.18084
1    -0.67825  1.93886  1.43533  0.76096 -0.14277   -0.71126 -0.21575
2    -0.46725  0.80523 -0.84732 -1.62090 -1.01450   -1.37983  0.40148
3    -0.14882 -0.80615 -0.01928  0.59042  0.58489   -1.37983 -1.18084
4     0.73545 -1.63340 -0.45174 -0.30172  1.30612   -0.21712 -0.21575
...       ...      ...      ...      ...      ...        ...      ...
1880 -1.19430  1.74091  1.88511  0.76096 -1.13788    0.88113  1.92173
1881 -0.24649  1.74091  0.58331  0.76096 -1.51840    0.88113  0.76540
1882  1.13281 -1.37639 -1.27553 -1.77200 -1.38502    0.52975 -0.52593
1883  0.91093 -1.92173  0.29338 -1.62090 -2.57309    1.29221  1.22470
1884 -0.46725  2.12700  1.65653  1.11406  0.

In [18]:
# Check unique target values for each substance before conversion
print("Unique target values before conversion:")
for column in drug_consumption.data.targets.columns:
    unique_values = drug_consumption.data.targets[column].unique()
    print(f"Unique values for {column}: {unique_values}")

# Create a new DataFrame to hold the converted values
converted_targets = drug_consumption.data.targets.copy()

# Convert target strings "CL#" to integers (1 to 6) using a for loop
for column in drug_consumption.data.targets.columns:
    for index in range(len(drug_consumption.data.targets)):
        converted_targets.loc[index, column] = int(drug_consumption.data.targets.loc[index, column][2])  # Extract the number and convert to int

# Replace the original targets with the converted DataFrame
drug_consumption.data.targets = converted_targets

# Check unique target values after conversion
print("\nUnique target values after conversion:")
for column in drug_consumption.data.targets.columns:
    unique_values = drug_consumption.data.targets[column].unique()
    print(f"Unique values for {column}: {unique_values}")

# Print targets
print(drug_consumption.data.targets)

Unique target values before conversion:
Unique values for alcohol: ['CL5' 'CL6' 'CL4' 'CL2' 'CL1' 'CL0' 'CL3']
Unique values for amphet: ['CL2' 'CL0' 'CL1' 'CL3' 'CL5' 'CL4' 'CL6']
Unique values for amyl: ['CL0' 'CL2' 'CL1' 'CL3' 'CL5' 'CL4' 'CL6']
Unique values for benzos: ['CL2' 'CL0' 'CL3' 'CL1' 'CL4' 'CL5' 'CL6']
Unique values for caff: ['CL6' 'CL5' 'CL4' 'CL3' 'CL0' 'CL1' 'CL2']
Unique values for cannabis: ['CL0' 'CL4' 'CL3' 'CL2' 'CL1' 'CL6' 'CL5']
Unique values for choc: ['CL5' 'CL6' 'CL4' 'CL0' 'CL3' 'CL2' 'CL1']
Unique values for coke: ['CL0' 'CL3' 'CL2' 'CL1' 'CL6' 'CL5' 'CL4']
Unique values for crack: ['CL0' 'CL1' 'CL2' 'CL3' 'CL5' 'CL4' 'CL6']
Unique values for ecstasy: ['CL0' 'CL4' 'CL1' 'CL3' 'CL2' 'CL6' 'CL5']
Unique values for heroin: ['CL0' 'CL1' 'CL2' 'CL3' 'CL5' 'CL6' 'CL4']
Unique values for ketamine: ['CL0' 'CL2' 'CL3' 'CL1' 'CL5' 'CL4' 'CL6']
Unique values for legalh: ['CL0' 'CL1' 'CL2' 'CL3' 'CL5' 'CL4' 'CL6']
Unique values for lsd: ['CL0' 'CL2' 'CL1' 'CL3' 'CL4'

### Exploratory Data Analysis:

In [20]:
# Assuming you have already cleaned the features as needed
X = drug_consumption.data.features  # Features
y = drug_consumption.data.targets    # Targets

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)

Training set size: (1508, 7)
Testing set size: (377, 7)


In [21]:
# Initialize the Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R² Score:", r2)

Mean Squared Error: 1.9264905632462008
R² Score: 0.1024075312889485


### Data Limitations:

### Questions for Reviewers: